# MNIST SI4 CNN

## Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical


import wandb
from wandb.keras import WandbMetricsLogger

## Load and format MNIST dataset

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255
x_test  = x_test.astype('float32')  / 255
x_train = x_train.reshape((60000, 28, 28, 1)) # 'channels_last' format
x_test  = x_test.reshape((10000, 28, 28, 1)) # 'channels_last' format
y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test,  10)

## Save validation data to CSV

In [3]:
np.savetxt('x_test.csv', x_test.reshape((x_test.shape[0], -1))[0:250], delimiter=',', fmt='%s') 
np.savetxt('y_test.csv', y_test[0:250], delimiter=',', fmt='%s')

## Build model

In [36]:
model = Sequential()
model.add(Input(shape=(28, 28, 1)))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=10, activation='softmax'))

# log NN details
wandb.init(
    project="Embedded-AI-Lab2-MINST",
    name="Kernel Size: 5x5 1 Pooling Layer | 2x2DConv (32 => 64) | 3 Epochs | Run 3",
    config=model.get_config()
)

model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])



batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
batch/categorical_accuracy,▁▄▅▆▆▆▇▇▇▇▇▇▇███████████████████████████
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▅▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/categorical_accuracy,▁▇█
epoch/epoch,▁▅█
epoch/learning_rate,▁▁▁
epoch/loss,█▂▁
epoch/val_categorical_accuracy,▁▇█
epoch/val_loss,█▁▁
batch/batch_step,5620


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 24, 24, 32)        832       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 12, 12, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 8, 8, 64)          51264     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 4, 4, 64)         0         
 g2D)                                                            
                                                                 
 flatten_8 (Flatten)         (None, 1024)              0         
                                                                 
 dense_8 (Dense)             (None, 10)               

## Train model

In [37]:
model.fit(x_train, y_train, 
    epochs=3, 
    validation_data=(x_test, y_test), 
    callbacks=[
        WandbMetricsLogger(log_freq=5)
    ])

Epoch 1/3
   4/1875 [..............................] - ETA: 32s - loss: 2.2619 - categorical_accuracy: 0.1406     

2023-01-13 08:26:56.561098: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1872/1875 [============================>.] - ETA: 0s - loss: 0.1386 - categorical_accuracy: 0.9586

2023-01-13 08:27:16.841624: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 23s 12ms/step - loss: 0.1384 - categorical_accuracy: 0.9587 - val_loss: 0.0419 - val_categorical_accuracy: 0.9874
Epoch 2/3
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0439 - categorical_accuracy: 0.9865 - val_loss: 0.0424 - val_categorical_accuracy: 0.9871
Epoch 3/3
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0310 - categorical_accuracy: 0.9899 - val_loss: 0.0288 - val_categorical_accuracy: 0.9908


## Evaluate model

In [38]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

313/313 - 2s - loss: 0.0288 - categorical_accuracy: 0.9908 - 2s/epoch - 7ms/step
 60/313 [====>.........................] - ETA: 0s

2023-01-13 08:28:06.294893: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 1s 3ms/step
tf.Tensor(
[[ 977    0    0    0    0    0    2    1    0    0]
 [   0 1131    2    0    0    0    1    1    0    0]
 [   1    0 1026    1    0    0    0    2    2    0]
 [   2    0    2 1003    0    1    0    0    1    1]
 [   0    0    0    0  972    0    1    0    0    9]
 [   1    0    0    8    0  878    2    0    2    1]
 [   5    2    1    0    2    1  943    0    4    0]
 [   0    1    5    1    0    0    0 1017    1    3]
 [   4    0    2    2    1    0    0    0  963    2]
 [   1    0    0    0    4    2    0    0    4  998]], shape=(10, 10), dtype=int32)


## Save trained model

In [39]:
model.save('./mnist_lenet5.h5')

wandb: Network error (ConnectionError), entering retry loop.
